# SMT Solver Comparative Analysis

All solvers have their available memory limited to 4GB. They are all given the same timeout value (the time limit for each check-sat). They are all given the same 25 test files. The test files are in the form of SMT2 files containing non-linear integer equations. Files are divided into 3 groups: problems involving 1 constant, problems involving 2 constants and problems involving 3 constants. Each of these groups contain sub-groups of quadratic and cubic problems. There is a mixture of both satisfiable and unsatisfiable equations. There is a slight bias for the frequency of unsatisfiable cubic equations as these were found to be the most demanding equations, thus making them especially suitable for testing performance.


In [8]:
import resource
import time
import csv
import ast
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import crt_solver.crt_solver as crt_solver
import cvc5_batchsolver.cvc5_solver as cvc5_solver
import z3_batchsolver.z3_solver as z3_solver


In [9]:
# Find root directory (CRTSolver-Report-Pipeline)
def find_root_directory():
    current = Path.cwd()
    # Search for pyproject.toml
    while current != current.parent:
        if (current / 'pyproject.toml').exists():
            return current
        current = current.parent

# Find root directory (CRTSolver-Report-Pipeline)
ROOT = find_root_directory()

# Set absolute paths from root directory
TESTS = ROOT / "tests"
RESULTS = ROOT / "results"


In [10]:
def generate_cactus_plot(solvers, memory, timeout):
    # Set size and layout
    plt.figure(figsize=(10, 6))
    plt.grid(True)
    plt.tight_layout()

    # Set title and labels
    plt.title(f"Speed of SMT Solvers with Non-Linear Integer Equations ({memory}GB, {timeout}s)")
    plt.xlabel("Problems solved (count)")
    plt.ylabel("Runtime (seconds)")

    for solver in solvers:
        update_cactus_plot(solver.get_solver_name())

    # Save and display final cactus plot
    plt.legend()
    plt.savefig("cactus.pdf", format="pdf", bbox_inches="tight")  
    plt.show()

def update_cactus_plot(solver_name):
    # Populate pandas dataframe with results stored in csv file
    file_name = RESULTS / f"results_{solver_name}.csv" # header row is automatically handled by pandas
    df = pd.read_csv(file_name)
    
    # Drop last row from results.csv (contains summary data)
    df = df[:-1]

    # Keep only equations that were successfully solved
    df["Result"] = df["Result"].apply(ast.literal_eval) # parse result column from string to list object
    df = df[~df["Result"].apply(lambda x: isinstance(x, list) and "UNKNOWN" in str(x[0][0]))] # ~ means NOT

    # Sort equations by ascending runtime
    df["Runtime (s)"] = df["Runtime (s)"].astype(float)
    df = df.sort_values(by="Runtime (s)").reset_index(drop=True) # drop original index row

    # Accumulate individual runtimes
    cumulative_runtimes = df["Runtime (s)"].cumsum()
    problem_indices = list(range(1, len(cumulative_runtimes) + 1))

    # Plot number of problems solved against cumulative runtime
    if solver_name == "CRTSolver (Integer Mode)":
        plt.plot(problem_indices, cumulative_runtimes,
            marker="+", linestyle="-", color="tab:blue", label=solver_name)
    elif solver_name == "CRTSolver (Bit-Vector Mode)":
        plt.plot(problem_indices, cumulative_runtimes,
            marker="*", linestyle="-", color="tab:red", label=solver_name)
    elif solver_name == "cvc5":
        plt.plot(problem_indices, cumulative_runtimes,
            marker="x", linestyle="-", color="tab:green", label=solver_name)
    elif solver_name == "Z3":
        plt.plot(problem_indices, cumulative_runtimes,
            marker="^", linestyle="-", color="tab:orange", label=solver_name)
    

In [12]:
def generate_bar_chart(solvers, memory, timeout):
    # Set size and layout
    plt.figure(figsize=(10, 6))
    plt.grid(True)
    plt.tight_layout()

    # Set title and labels
    plt.title(f"Success Rates of SMT Solvers with Non-Linear Integer Equations ({memory}GB, {timeout}s)")
    plt.xlabel("SMT Solver")
    plt.ylabel("Success Rate (%)")

    solver_names = []
    success_rates = []
    for solver in solvers:
        solver_name = solver.get_solver_name()
        accuracy = calculate_success_rate(solver_name)
        solver_names.append(solver_name)
        success_rates.append(accuracy)

    # Return positions of solvers
    x_positions = range(len(solver_names))

    # Create bar chart
    if len(x_positions) == 3:
        plt.bar(x_positions, success_rates, width=0.5, color=["tab:red", "tab:green", "tab:orange"])
    else:
        plt.bar(x_positions, success_rates, width=0.5, color=["tab:blue", "tab:red", "tab:green", "tab:orange"])

    # Label positions with solver names
    plt.xticks(x_positions, solver_names)
    plt.show()

def calculate_success_rate(solver_name):
    file_name = RESULTS / f"results_{solver_name}.csv"
    total_count = 0
    unknown_count = 0

    with open(file_name, newline="") as file:
        reader = csv.reader(file)
        rows = list(reader) # return list of rows
        for row in rows[1:-1]: # exclude first and last rows
            if row:
                model = row[5]
                total_count += 1
                if "UNKNOWN (TIMEOUT)" in model or "UNKNOWN (ERROR)" in model:
                    unknown_count += 1

    success_rate = ((total_count - unknown_count) / total_count) * 100
    return success_rate


In [ ]:
def execute_comparison(memory, timeout, use_crt_int):
    # Set control variables
    CHECK_SAT_TLIMIT = timeout # time limit for each check-sat call across all solvers
    MEM_LIMIT = memory # memory limit across all solvers
    timeout_s = int(timeout) / 1000

    # Instantiate solver classes
    solvers = []

    if use_crt_int:
        crtINT = crt_solver.CRTSolver(CHECK_SAT_TLIMIT, tests_dir=TESTS, no_bitvectors=True)
        solvers.append(crtINT)

    crtBV = crt_solver.CRTSolver(CHECK_SAT_TLIMIT, tests_dir=TESTS)
    solvers.append(crtBV)
    cvc5 = cvc5_solver.cvc5Solver(CHECK_SAT_TLIMIT, tests_dir=TESTS)
    solvers.append(cvc5)
    z3 = z3_solver.Z3Solver(CHECK_SAT_TLIMIT, tests_dir=TESTS)
    solvers.append(z3)

    # Execute solvers
    for solver in solvers:
        execute_solver(solver, MEM_LIMIT)

    # Generate cactus plot for speed comparisons
    generate_cactus_plot(solvers, MEM_LIMIT, timeout_s)

    # Generate bar chart for accuracy comparisons
    generate_bar_chart(solvers, MEM_LIMIT, timeout_s)

def execute_solver(solver, memory):
    solver_name = solver.get_solver_name()

    # Set memory limit
    set_memory_limit(memory)
    print(f"Executing {solver_name} with {memory}GB memory")
    
    try:
        start_time = time.time()
        solver.execute()
        duration = time.time() - start_time
        print(f"Completed solving in {duration}s")
    except MemoryError as e:
        print(f"Stopping execution due to MemoryError: exceeded memory limit of {memory}GB")

def set_memory_limit(gb):
    # Raising memory limit requires restarting the session

    # Convert gigabytes to bytes
    # 1gb = 1024^3 bytes
    GIGABYTE = 1024 ** 3
    max_memory = int(gb * GIGABYTE) # resource.setrlimit needs integers

    # Set new memory limits - soft and hard limits are set to same value
    # Soft limit = currently allocated, hard limit = maximum allocable
    resource.setrlimit(resource.RLIMIT_AS, (max_memory, max_memory))


In [16]:
# Compare with 4GB memory and 0.001 seconds timeout (for testing)
execute_comparison(4, "1", True)

Executing CRTSolver (Integer Mode) with 4GB memory
Reading file: /home/matvm/CRTSolver-Report-Pipeline/tests/1var/2deg/theater.smt2
Attempting to solve with mod 2
UNKNOWN (TIMEOUT)

Reading file: /home/matvm/CRTSolver-Report-Pipeline/tests/1var/2deg/cat.smt2
Attempting to solve with mod 2
Retrieving candidates for mod 2
x: 0
Attempting to solve with candidates:
x: 0
Candidate UNSAT
Attempting to solve with candidates:
x: (- 2)
Candidate UNSAT
Attempting to solve with candidates:
x: 2
Candidate UNSAT
Attempting to solve with candidates:
x: (- 4)
Candidate UNSAT
Attempting to solve with candidates:
x: 4
Candidate UNSAT
Attempting to solve with mod 3
UNKNOWN (TIMEOUT)

Reading file: /home/matvm/CRTSolver-Report-Pipeline/tests/1var/2deg/dog.smt2
Attempting to solve with mod 2
UNKNOWN (TIMEOUT)

Reading file: /home/matvm/CRTSolver-Report-Pipeline/tests/1var/2deg/house.smt2
Attempting to solve with mod 2
UNKNOWN (TIMEOUT)

Reading file: /home/matvm/CRTSolver-Report-Pipeline/tests/1var/2deg/e

KeyboardInterrupt: 

In [ ]:
# Compare with 4GB memory and 30 seconds timeout
#execute_comparison(4, "30000", False)


In [ ]:
# Compare with 4GB memory and 10 seconds timeout
#execute_comparison(4, "10000", True)

In [ ]:
# Compare with 4GB memory and 5 seconds timeout
#execute_comparison(4, "5000", True)